In [ ]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
#from dataset_loader import load_dataset

#tf.enable_eager_execution()
INPUT_WIDTH = 64
INPUT_HEIGHT = 65

INPUT_SIZE = 64

INPUT_CHANNELS = 1

NUM_CLASSES = 5

LEARNING_RATE = 0.0005   # Original value: 0.01
MOMENTUM = 0.9
KEEP_PROB = 1

EPOCHS = 2000
BATCH_SIZE = 64

NUM_PARRALLEL_THREADS = 8

train_sub_dir = '/data/image_format_small_dataset/*/*'
val_sub_dir = '/data/image_format_small_dataset_val/*/*'
test_sub_dir = '/data/image_format_small_dataset_test/*/*'

frequency_train_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_cogisen_updated/*/*/*.csv'
frequency_val_sub_dir = '/data/image_format_small_dataset_val_cogisen_updated/*/*/*.csv'
frequency_test_sub_dir = '/data/image_format_small_dataset_test_cogisen_updated/*/*/*.csv'


home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(img_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True,num_parallel_calls=NUM_PARRALLEL_THREADS)


val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True,num_parallel_calls=NUM_PARRALLEL_THREADS)

test_dataset = load_dataset(img_dir= test_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True,num_parallel_calls=NUM_PARRALLEL_THREADS)

experiment_no = 1
#logdir = './log/alexnet/FULL_DATA'+str(experiment_no)
#logdir = './log/frequency_data/grayscale+normalized' +str(experiment_no)

#logdir = './log/rubbish'
logdir = './log/simple_model3'
weight_dir = './weights/experiment '+str(experiment_no)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'XLA_GPU']

In [ ]:
get_available_gpus()

In [ ]:

handle = tf.placeholder(tf.string, shape=[])


training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
training_init_op = training_iterator.make_initializer(training_dataset)




In [ ]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()

In [ ]:
#with tf.device('/device:XLA_GPU:0'):
alexnet = Simple_Model(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES,input_channels=INPUT_CHANNELS)

In [ ]:
with tf.device('/device:XLA_GPU:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        print('Training dataset...')
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        training_string_handle =  sess.run(training_iterator.string_handle())

        summary_operation = tf.summary.merge_all()

        for epoch in range(EPOCHS):
            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            while True:
                try:
                    _, accuracy,summary,predictions,Y= sess.run([alexnet.training_operation,alexnet.accuracy_operation,summary_operation,alexnet.predictions,alexnet.Y],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob,handle:training_string_handle})
                   
                
                except tf.errors.OutOfRangeError:
                    file_writer.add_summary(summary, epoch)
                    print("accuracy for epoch {0}  :  {1}".format(epoch,accuracy))
                   
                    break

        alexnet.save(sess,)
            
            

In [ ]:

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(val_init_op)
    val_string_handle =  sess.run(val_iterator.string_handle())
    val_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:val_string_handle})
    

    print("accuracy for validation set :  {0}".format(val_accuracy))



In [ ]:

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('Testing dataset...')
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    sess.run(test_init_op)
    test_string_handle =  sess.run(test_iterator.string_handle())
    test_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:test_string_handle})
    

    print("accuracy for test set :  {0}".format(test_accuracy))



In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())